In [1]:
%matplotlib inline
import copy
from qutip import *
import numpy as np
import math, cmath
import matplotlib.pyplot as plt
from tqdm import tqdm

import os

from IPython.display import Markdown, display

In [32]:
TIMESTEPS = 2000
TIMEDELTA = 1e-02 
OMEGA = 0.1  # Strength of Interaction

D = 10

th = OMEGA * TIMEDELTA
alpha = 0.5
phi = np.pi/2
ga = 2*alpha**2
gb = (1-alpha)**2(1-np.cos(phi))

<>:11: SyntaxWarning: 'int' object is not callable; perhaps you missed a comma?
<>:11: SyntaxWarning: 'int' object is not callable; perhaps you missed a comma?


AttributeError: 'Qobj' object has no attribute 'pi'

In [29]:

# Creation and Annihilation Operators
ad = create(D)
a = destroy(D)
# Number Operators
n = num(D)
np = n+1

# Bosonic Operators
C = (th*(2*np).sqrtm()).cosm()
Cp = (th*(2*n).sqrtm()).cosm()
S = C = ad*(th*(2*np).sqrtm()).sinm()*((2*np).sqrtm()).inv()

In [31]:
def commutator(A, B):
    return A*B - B*A

def anticommutator(A, B):
    return A*B + B*A

def dissipator(X: Qobj, operator: Qobj):
    return X.dag() * operator * X - 1/2 * anticommutator(X.dag()*X, operator)
        